### current version

In [5]:
import sys
sys.path.append(".")

In [1]:
from analyze import *
#import analyze
# import analyze_original
# import analyze_new_prompt
from modules.llm_eval.consistency_eval import *

In [2]:
repo_path = '../../data/raw/openja/lightfm'
checklist_path_v1 = 'archive/checklist_20240527.csv'
checklist_path_v2 = '../../checklist/checklist.csv'
report_output_path_html = '../../report/evaluation_report.html'
report_output_path_pdf = '../../report/evaluation_report.pdf'

In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
checklist_v1 = Checklist(checklist_path_v1, checklist_format=ChecklistFormat.CSV)
checklist_v2 = Checklist(checklist_path_v2, checklist_format=ChecklistFormat.CSV)
repo = Repository(repo_path)
prompt_format = EvaluationPromptFormat()

evaluator_v1 = PerFileTestEvaluator(llm, prompt_format=prompt_format, repository=repo, checklist=checklist_v1)
evaluator_v2 = PerFileTestEvaluator(llm, prompt_format=prompt_format, repository=repo, checklist=checklist_v2)

In [5]:
eval_evaluator = ConsistencyEvaluator()

eval_evaluator.evaluate(models=[
    {'name': '1. original', 'model': evaluator_v1},
    {'name': '2. after rewording Requirement', 'model': evaluator_v2},
    #{'name': '3. after removing Title from prompt and minor prompt fix', 'model': evaluator_v3},
    # {'name': '4. add checklist Explanation into prompt', 'model': evaluator_v4},
]
, num_test_runs=5) # TODO: recommended 20 ~ 30

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:21<00:00, 11.67s/it]


In [6]:
eval_evaluator.get_completeness_score_dist()

test_no,1,2,3,4,5
model_name,,,,,
1. original,0.500000,0.428571,0.428571,0.428571,0.428571
2. after rewording Requirement,0.642857,0.571429,0.500000,0.571429,0.500000


In [7]:
completeness_score_var = pd.DataFrame(eval_evaluator.get_completeness_score_dist().var(axis=1), columns=['sample variance'])
completeness_score_var

,sample variance
model_name,
1. original,0.001020
2. after rewording Requirement,0.003571


In [8]:
checklist_item_title = pd.DataFrame(checklist_v2.get_all_tests())[['ID', 'Title']]
checklist_item_title

,ID,Title
0,2.1,Ensure Data File Loads as Expected
1,3.2,Data in the Expected Format
2,3.5,Check for Duplicate Records in Data
3,4.2,Verify Data Split Proportion
4,5.3,Ensure Model Output Shape Aligns with Expectation
5,6.1,Verify Evaluation Metrics Implementation
6,6.2,Evaluate Model's Performance Against Thresholds


In [9]:
# Human evaluated answer
human_answer = pd.DataFrame([{'ID': '2.1', "human_answer": 1.0}, 
          {'ID': '3.2', "human_answer": 0.5},
          {'ID': '3.5', "human_answer": 0.0},
          {'ID': '4.2', "human_answer": 1.0},
          {'ID': '5.3', "human_answer": 0.5},
          {'ID': '6.1', "human_answer": 1.0},
          {'ID': '6.2', "human_answer": 1.0},
         ])
human_answer

,ID,human_answer
0,2.1,1.0
1,3.2,0.5
2,3.5,0.0
3,4.2,1.0
4,5.3,0.5
5,6.1,1.0
6,6.2,1.0


In [10]:
consistency_dist = eval_evaluator.get_consistency_dist().reset_index().drop(columns=['consistency'])
consistency_dist = consistency_dist.merge(human_answer, how='left', on='ID')
consistency_dist = consistency_dist.merge(checklist_item_title, how='left', on='ID')
consistency_dist

,model_name,ID,1,2,3,4,5,human_answer,Title
0,1. original,2.1,0.0,0.0,0.0,0.0,0.0,1.0,Ensure Data File Loads as Expected
1,1. original,3.2,0.5,0.5,0.5,0.5,0.5,0.5,Data in the Expected Format
2,1. original,3.5,0.0,0.0,0.0,0.0,0.0,0.0,Check for Duplicate Records in Data
3,1. original,4.2,1.0,0.5,1.0,1.0,1.0,1.0,Verify Data Split Proportion
4,1. original,5.3,0.0,0.0,0.0,0.0,0.0,0.5,Ensure Model Output Shape Aligns with Expectation
5,1. original,6.1,1.0,1.0,0.5,0.5,0.5,1.0,Verify Evaluation Metrics Implementation
6,1. original,6.2,1.0,1.0,1.0,1.0,1.0,1.0,Evaluate Model's Performance Against Thresholds
7,2. after rewording Requirement,2.1,1.0,1.0,1.0,1.0,1.0,1.0,Ensure Data File Loads as Expected
8,2. after rewording Requirement,3.2,0.5,1.0,0.5,0.5,0.5,0.5,Data in the Expected Format
9,2. after rewording Requirement,3.5,0.0,0.0,0.0,0.0,0.0,0.0,Check for Duplicate Records in Data


In [4]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
checklist_v1 = Checklist(checklist_path_v1, checklist_format=ChecklistFormat.CSV)
checklist_v2 = Checklist(checklist_path_v2, checklist_format=ChecklistFormat.CSV)
extractor = PythonTestFileExtractor(Repository(repo_path))

evaluator_v1 = analyze_original.TestEvaluator(llm, extractor, checklist_v1)
evaluator_v2 = analyze_original.TestEvaluator(llm, extractor, checklist_v2)
evaluator_v3 = analyze.TestEvaluator(llm, extractor, checklist_v2)
# evaluator_v4 = analyze_new_prompt.TestEvaluator(llm, extractor, checklist_v2)

In [5]:
eval_evaluator = ConsistencyEvaluator()

eval_evaluator.evaluate(models=[
    {'name': '1. original', 'model': evaluator_v1},
    {'name': '2. after rewording Requirement', 'model': evaluator_v2},
    {'name': '3. after removing Title from prompt and minor prompt fix', 'model': evaluator_v3},
    # {'name': '4. add checklist Explanation into prompt', 'model': evaluator_v4},
]
, num_test_runs=5) # TODO: recommended 20 ~ 30

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [01:23<00:00, 11.97s/it]


In [27]:
# pd.set_option('display.max_colwidth', None)
# eval_evaluator.evaluation_reports['report'][12] #[11]

In [23]:
# pd.reset_option('display.max_colwidth')

In [22]:
eval_evaluator.get_completeness_score_dist()

test_no,1,2,3,4,5
model_name,,,,,
1. original,0.428571,0.285714,0.357143,0.357143,0.357143
2. after rewording Requirement,0.642857,0.500000,0.642857,0.500000,0.571429
3. after removing Title from prompt and minor prompt fix,0.500000,0.714286,0.642857,0.714286,0.571429


In [23]:
completeness_score_var = pd.DataFrame(eval_evaluator.get_completeness_score_dist().var(axis=1), columns=['sample variance'])
completeness_score_var

,sample variance
model_name,
1. original,0.002551
2. after rewording Requirement,0.005102
3. after removing Title from prompt and minor prompt fix,0.008673


In [24]:
checklist_item_title = pd.DataFrame(checklist_v2.get_all_tests())[['ID', 'Title']]

In [25]:
# Human evaluated answer
human_answer = pd.DataFrame([{'ID': '2.1', "human_answer": 1.0}, 
          {'ID': '3.2', "human_answer": 0.5},
          {'ID': '3.5', "human_answer": 0.0},
          {'ID': '4.2', "human_answer": 1.0},
          {'ID': '5.3', "human_answer": 0.5},
          {'ID': '6.1', "human_answer": 1.0},
          {'ID': '6.2', "human_answer": 1.0},
         ])
human_answer

,ID,human_answer
0,2.1,1.0
1,3.2,0.5
2,3.5,0.0
3,4.2,1.0
4,5.3,0.5
5,6.1,1.0
6,6.2,1.0


In [26]:
consistency_dist = eval_evaluator.get_consistency_dist().reset_index().drop(columns=['consistency'])
consistency_dist = consistency_dist.merge(human_answer, how='left', on='ID')
consistency_dist = consistency_dist.merge(checklist_item_title, how='left', on='ID')
consistency_dist

,model_name,ID,1,2,3,4,5,human_answer,Title
0,1. original,2.1,1.0,0.0,0.0,0.0,0.0,1.0,Ensure Data File Loads as Expected
1,1. original,3.2,0.0,0.0,0.5,0.5,0.5,0.5,Data in the Expected Format
2,1. original,3.5,0.0,0.0,0.0,0.0,0.0,0.0,Check for Duplicate Records in Data
3,1. original,4.2,0.5,0.5,0.5,0.5,0.5,1.0,Verify Data Split Proportion
4,1. original,5.3,0.0,0.0,0.0,0.0,0.0,0.5,Ensure Model Output Shape Aligns with Expectation
5,1. original,6.1,0.5,0.5,0.5,0.5,0.5,1.0,Verify Evaluation Metrics Implementation
6,1. original,6.2,1.0,1.0,1.0,1.0,1.0,1.0,Evaluate Model's Performance Against Thresholds
7,2. after rewording Requirement,2.1,1.0,1.0,1.0,1.0,1.0,1.0,Ensure Data File Loads as Expected
8,2. after rewording Requirement,3.2,0.5,0.0,0.5,0.5,0.5,0.5,Data in the Expected Format
9,2. after rewording Requirement,3.5,0.0,0.0,0.0,0.0,0.0,0.0,Check for Duplicate Records in Data


In [27]:
# consistency_dist[['model_name', 'ID', 'Title', 1, 2, 3, 4, 5, 'human_answer']]

In [28]:
# !conda install matplotlib -y

In [29]:
# import matplotlib

# temp[['ID', 1, 2, 3, 4, 5]].set_index(['ID']).T.plot(
#     kind='box',
#     # axis=0,
# )

In [30]:
# temp = consistency_dist[consistency_dist.model_name == '2. after prompt engineering'].copy().reset_index()
# temp['repo'] = 'LightFM'
# temp[['repo', 'ID', 1, 2, 3, 4, 5]].set_index(['repo', 'ID']).T

In [31]:
# eval_evaluator.evaluation_reports['report'][0]['Function References'][0]

### last week's version 

In [32]:
# from archive.analyze import TestEvaluator as TestEvaluatorPrev
# from archive.modules.llm_eval.consistency_eval import ConsistencyEvaluator as ConsistencyEvaluatorPrev

In [33]:
# evaluator_prev = TestEvaluatorPrev('../../data/raw/openja/lightfm_demo')
# evaluator_prev.load_checklist('archive/checklist_sys.csv')

In [34]:
# eval_prev_evaluator = ConsistencyEvaluatorPrev()

# eval_prev_evaluator.evaluate(
#     models=[{'name': 'model_no1', 'model': evaluator_prev}], 
#     num_test_runs=10, 
#     version_before_refactoring=True)

In [35]:
# eval_prev_evaluator.get_completeness_score_dist()

In [36]:
# eval_prev_evaluator.get_completeness_score_dist().iloc[0].var()

In [37]:
# eval_prev_evaluator.get_consistency_dist()

### F-test to examine the variance difference

In [53]:
var_curr = completeness_score_var[completeness_score_var.index == '2. after rewording Requirement'].iloc[0, 0]
var_prev = completeness_score_var[completeness_score_var.index == '1. original'].iloc[0, 0]

In [52]:
var_curr

0.0051020408163265345

In [54]:
df_curr = eval_evaluator.get_completeness_score_dist().shape[1] - 1
df_prev = eval_evaluator.get_completeness_score_dist().shape[1] - 1

In [55]:
if var_prev < var_curr:
    F = var_curr / var_prev 
else:
    F = var_prev / var_curr
# F = var_prev / var_curr 
F

2.000000000000002

In [ ]:
# !conda install scipy -y

In [56]:
import scipy
tail = 2
alpha = 0.05 #Or whatever you want your alpha to be.
p_value = 1 - scipy.stats.f.cdf(F, df_prev, df_curr) 

print(f"p-value: {p_value}")
print()

print(f"{tail}-tail test:")
if p_value < alpha / 2:
    print("  Reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))")
else:
    print("  Failed to reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))")

p-value: 0.25925925925925886

2-tail test:
  Failed to reject the null hypothesis: Var(Completeness_Score(Current Version)) == Var(Completeness_Score(Last Week Version))


### reference
https://www.itl.nist.gov/div898/handbook/eda/section3/eda359.html  
https://www.statisticshowto.com/probability-and-statistics/hypothesis-testing/f-test/#:~:text=F%20Test%20to%20Compare%20Two%20Variances,-A%20Statistical%20F&text=If%20the%20variances%20are%20equal,when%20running%20an%20F%20Test  
https://stackoverflow.com/questions/21494141/how-do-i-do-a-f-test-in-python